In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
import pickle

with open('alltrends_123.pickle', 'rb') as handle:
    alltrends_12 = pickle.load(handle)
    
with open('NPDdata.pickle', 'rb') as handle:
    NPDdata = pickle.load(handle)

    
with open('gamematches123.pickle', 'rb') as handle:
    gamematches123 = pickle.load(handle)

gamelist = []

for i in range(0,len(gamematches123)):
    gamelist.append(gamematches123[i][0])
    
NPDdata = NPDdata.rename(index=str, columns={"Overall Top 10 Video Game Titles": 'July 2009 NPD Games Sales'})


In [3]:
#converts month to the following. Added one to evaluate previous month from 1st day of following month
monthdic = {'January':'02',
'February':'03',
'March': '04',
'April': '05',
'May': '06',
'June': '07',
'July': '08',
'August': '09',
'September': '10',
'October': '11',
'November': '12',
'December': '01'}

In [4]:
monthcol = list(NPDdata.columns.values) 

In [5]:
dates = []
for i in range(0,len(monthcol)):
    [month,year] = [monthcol[i].split()[0],monthcol[i].split()[1]]
    dates.append([month,year])

monthlist = [] #mark days by first of following month
for i in range(0,len(dates)):
    if dates[i][0] == 'December':
        day = str(int(dates[i][1])+1) + '-' + monthdic[dates[i][0]] + '-01'
    else:
        day = dates[i][1] + '-' + monthdic[dates[i][0]] + '-01'
    monthlist.append(day)

In [6]:
NPDdata.columns = monthlist

In [7]:
def xmonthsago (xmonths, date,datelist): #pulls out dates from datelist from x months before date
    #YYYY-MM-DD
    thedays = []
    year_i = int(date[:4])
    month_i = int(date[5:7])
    m = month_i - xmonths
    y = year_i
    while m<1:
        m = m+12
        y = y -1
    if m < 10:
        str_m = '0'+str(m)
    else: 
        str_m = str(m)
    ym = str(y)+'-'+str_m
    for i in range(0,len(datelist)):
        if ym in datelist[i]:
            thedays.append(datelist[i])
    return thedays
        
    
    

In [8]:
datelist = list(alltrends_12.index.values)

In [9]:
def getfeature (data, game, thedays): #retrieves max in trend data restrcted to dates from list: thedays
    x = data.filter(items=thedays,axis=0).max()[game]   
    return x
    
    

## Creating Features
The features we are using are days ago since absolute maximum (as a stand in for release date).
The value of the absolute maximum and the peak searches over the last 3 months.

The 6th feature is a binary output of whether or not the absolute maximum was between 5 to 55 days ago. This may seem redundant with the first feature, but when training the model, the new features for the pairwise comparison will be comprised of the differences of these features. With only  the difference of feature 0, 5 days - 35 days would be considered the same as 35 days - 65 days. But as shown in the data visualization file, games released <5 days ago rarely chart since they haven't had enough days to sell, while games that came out about 1-2 months ago (but more than 5 days) tend to do well.

In [10]:
#Features: 0: days since peak (negative if in future), 1: value of max peak (0 if hasn't occured yet), 2-4: max peak for 1,2,3 month(s) ago
X = np.zeros((len(monthlist),len(gamelist),6)) #month, game, features
date_format = "%Y-%m-%d"

for i in range(0,len(monthlist)):
    thedays1 = xmonthsago(1, monthlist[i], datelist)
    thedays2 = xmonthsago(2, monthlist[i], datelist)
    thedays3 = xmonthsago(3, monthlist[i], datelist)
    for j in range(0,len(gamelist)):
        maxdate = alltrends_12[gamelist[j]].idxmax()[:10] 
        x1 = (datetime.strptime(monthlist[i], date_format) - datetime.strptime(maxdate, date_format)).days
        if x1 < 0:
            x2 = 0
        else:
            x2 = alltrends_12.max()[gamelist[j]]
        X[i,j,0] = x1
        X[i,j,1] = x2
        X[i,j,2] = getfeature(alltrends_12,gamelist[j],thedays1)
        X[i,j,3] = getfeature(alltrends_12,gamelist[j],thedays2)
        X[i,j,4] = getfeature(alltrends_12,gamelist[j],thedays3)
        if x1 >= 5 and x1 <= 55: #Did the game come out recently? (but not too recently)
            X[i,j,5] = 1
        else: 
            X[i,j,5] = -1
        #X[month,game,feature index]
            
               

In [11]:
import pickle
with open('X.pickle','wb') as handle:
    pickle.dump(X,handle,protocol=pickle.HIGHEST_PROTOCOL)
    
with open('monthlist.pickle','wb') as handle:
    pickle.dump(monthlist,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open('gamelist.pickle','wb') as handle:
    pickle.dump(gamelist,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
#recording each month the rank of each game. If unranked y = 100
Y = np.ones((len(monthlist),len(gamelist)))
Y = Y * 100

for i in range(0,len(monthlist)):
    for k in range(1,20):
        rank_str = str(k)+'.0'
        game_r = NPDdata.loc[rank_str][monthlist[i]]
        if type(game_r) == str:
            j = gamelist.index(game_r)
            Y[i,j] = k #[month,game]
        

In [13]:
with open('Y.pickle','wb') as handle:
    pickle.dump(Y,handle,protocol=pickle.HIGHEST_PROTOCOL)